# Program memory: pointer arithmetic; malloc
_COSC 208, Introduction to Computer Systems, 2023-02-22_

## Announcements
* Modified TA hours tonight: 4:30pm-6:30pm
* Programming project 2 due Thursday, March 2 @ 11pm

## Outline
* Warm-up
* Pointer arithmetic
* Pointers as return values
* Program memory
* malloc

## Warm-up
* Q1: _What does the following program output?_

In [7]:
#include <stdio.h>
void update(char *x, char *y) {
    char *z = &x[3];
    *x = 's';
    *y = 'b';
    *z = 'k';
}

int main() {
    char one[] = "rice";
    char two[] = "neat";
    printf("%c %c\n", *one, *two);
    char *ptr1 = one;
    char *ptr2 = &two[3];
    *ptr1 = *two;
    *ptr2 = 'r';
    printf("%s %s\n", one, two);
    update(one, two);
    printf("%s %s\n", one, two);
}

r n
nice near
sick bear


<p style="height:5em;"></p>

🛑 **STOP here** after completing the above question; if you have extra time please **skip ahead** to the extra practice.

<div style="page-break-after:always;"></div>

## Pointer arithmetic

* Adding `n` to a pointer without dereferencing determines the address of the element `n` indices away
* Pointer does not change, unless the new address is assigned to the pointer

* Example

In [18]:
#include <stdio.h>
int main() {
    int nums[] = {2,4,8,16,32,64,128};
    
    int *ptr = nums + 2;
    printf("%d %d\n", *ptr, *nums);
    ptr = ptr + 2;
    printf("%d %d\n", *ptr, *nums);
    // nums = nums + 2; // Not allowed; array variable must point to 0th element
    
    int val = *(nums + 2);
    printf("%d %d\n", val, *nums);
    val = *(ptr + 2);
    printf("%d %d\n", val, *ptr);
    
    nums[2] = 0;
    printf("%d %d\n", *nums, nums[2]);
    *(nums + 2) = -1;
    printf("%d %d\n", *nums, nums[2]);
}

8 2
32 2
8 2
128 32
2 0
2 -1


* Q2: _What does the following program output?_

In [4]:
#include <stdio.h>
int main() {
    char phrase[] = "the cat sits";
    char *mysteryA = phrase + 4;
    char *mysteryB = mysteryA + 4;
    printf("%c%c\n", *mysteryA, *mysteryB);
    *(mysteryA + 2) = 'p';
    *mysteryB = 'f';
    printf("%s\n", phrase);
    mysteryA--;
    mysteryB += 2;
    *mysteryB = 'n';
    printf("red%s\n", mysteryA);
}

cs
the cap fits
red cap fins


<p style="height:6em;"></p>

* Q3: _Write a function called `sum` which takes an array of integers and a length and returns the sum of the numbers. The function **must** use a pointer to iterate over the array._

In [2]:
#include <stdio.h>
int sum(int *nums, int len) {
    int *curr = nums;
    int total = 0;
    for (int i = 0; i < len; i++) {
        total += *curr;
        curr++;
    }
    return total;
}
int main() {
    int nums[] = {1, 2, 4, 8};
    printf("%d\n", sum(nums, 4));
}

15


<div style="page-break-after:always;"></div>

## Pointers as return values
* _What happens?_

In [3]:
#include <stdio.h>
int *one() {
    int x = 1;
    int *p = &x;
    return p;
}
int main() {
    int *q = one();
    printf("%d\n", *q);
}

1


* `q` points to a variable that no longer exists!
* _So, how can I return a pointer from a function?_ — dynamically allocate memory on the heap!

## Program memory
* Memory layout
    ```
    ┏━━━━━━━━━━━━━━━━━━┓ 0
    ┃ Operating system ┃
    ┣━━━━━━━━━━━━━━━━━━┫
    ┃       Code       ┃ <- Function instructions
    ┣━━━━━━━━━━━━━━━━━━┫
    ┃       Data       ┃ <- Global variables
    ┣━━━━━━━━━━━━━━━━━━┫
    ┃       Heap       ┃ <- Dyanmically allocated memory
    ┃         ▼        ┃    Grows toward higher memory addresses
    ┣━━━━━━━━━━━━━━━━━━┫    
    ┃                  ┃
    ┣━━━━━━━━━━━━━━━━━━┫ 
    ┃         ▲        ┃    Grows toward lower memory addresses
    ┃       Stack      ┃ <- Local variables and parameters
    ┗━━━━━━━━━━━━━━━━━━┛ Max address
    ```
    * Stack consists of stack frames --- add a frame when a function is called, remove a frame when a function returns
* Variable storage
    * Local variables and parameters and stored on the stack --- in the frame for the function in which they are declared
    * Global variables are stored in the data section
* Memory allocation
    * Code and data — automatically allocated with a program starts
    * Stack — automatically allocated when a function is called; automatically deallocated when a function returns
    * Heap memory — explicitly allocated and freed by a program

## malloc

* `void* malloc(unsigned int size)`
* Memory allocated on the heap persists until explicitly freed
* When to malloc?
    * When the amount of space required is not known until runtime
    * When a value must remain in memory even after returning from a function
* How much to malloc?
    * Use `sizeof` and a type: e.g., `sizeof(int)`
    * How much to malloc for an array? — multiply sizeof(type) by number of elements in array

* Q4: _Write a function called `duplicate` that takes a string (i.e., an array of `char`) as a parameter and returns a copy of that string stored on the heap._

In [ ]:
#include <stdlib.h>
#include <string.h>
char *duplicate(char orig[]) {
    char *copy = malloc(sizeof(char) * (strlen(orig) + 1));
    for (int i = 0; i <= strlen(orig); i++) {
        copy[i] = orig[i];
    }
    // Could replace for loop with: strcpy(copy, orig);
    return copy;
}
// Testing
#include <assert.h>
int main() {
    char *copy = duplicate("abc 123");
    assert(strcmp(copy, "abc 123") == 0);
}

<p style="height:20em;"></p>

* Q5: _Write a function called `range` that behaves similar to the `range` function in Python. Your function should take an unsigned integer (`length`) as a parameter, and return a dynamically allocated array with `length` unsigned integers. The array should be populated with the values 0 through `length-1`._

In [10]:
#include <stdlib.h>
unsigned int *range(unsigned int length) {
    unsigned int *nums = malloc(sizeof(unsigned int) * length);
    for (int i = 0; i < length; i++) {
        nums[i] = i;
    }
    return nums;
}
// Testing
#include <assert.h>
int main() {
    unsigned int *result = range(3);
    assert(result[0] == 0 && result[1] == 1 && result[2] == 2);
}

<div style="page-break-after:always;"></div>

## Extra practice

* Q6: _Write a function called `generate_password` that takes an unsigned integer (`length`) as a parameter, and returns a dynamically allocated array of with `length` randomly selected characters (e.g., uppercase letters, lowercase letters, digits, symbols). Your function should use the `rand()` function from the C standard library, which returns a pseudo-random integer in the range 0 to `RAND_MAX`._

In [13]:
#include <stdlib.h>
char *generate_password(unsigned int length) {
    char *password = malloc(sizeof(char) * (length + 1));
    for (int i = 0; i < length; i++) {
        password[i] = (rand() % ('~' - '!')) + '!';
    }
    password[length] = '\0';
    return password;
}
// Testing
#include <stdio.h>
int main() {
    printf("%s\n", generate_password(8));
}

d:8#U2`t


<p style="height:20em;"></p>

* Q7: _Write a function called `lengths` that takes an array of strings and the number of elements in the array and returns an array of integers containing the length of each string._

In [17]:
#include <stdlib.h>
#include <string.h>
int *lengths(char *strs[], int count) {
    int *lens = malloc(sizeof(int) * count);
    for (int i = 0; i < count; i++) {
        lens[i] = strlen(strs[i]);
    }
    return lens;
}
// Testing
#include <assert.h>
int main() {
    char *strs[] = {"abc", "12345", "do re mi"};
    int *lens = lengths(strs, 3);
    assert(lens[0] == 3 && lens[1] == 5 && lens[2] == 8);
}